# Análisis del riesgo de incumplimiento de los prestatarios

Tu proyecto consiste en preparar un informe para la división de préstamos de un banco. Deberás averiguar si el estado civil y el número de hijos de un cliente tienen un impacto en el incumplimiento de pago de un préstamo. El banco ya tiene algunos datos sobre la solvencia crediticia de los clientes.

Tu informe se tendrá en cuenta al crear una **puntuación de crédito** para un cliente potencial. La **puntuación de crédito** se utiliza para evaluar la capacidad de un prestatario potencial para pagar su préstamo.

## Abre el archivo de datos y mira la información general. 

In [1]:
# Cargar todas las librerías
import pandas as pd
# Carga los datos
data_bank = pd.read_csv('/datasets/credit_scoring_eng.csv')

## Ejercicio 1. Exploración de datos

**Descripción de los datos**
- `children` - el número de hijos en la familia
- `days_employed` - experiencia laboral en días
- `dob_years` - la edad del cliente en años
- `education` - la educación del cliente
- `education_id` - identificador de educación
- `family_status` - estado civil
- `family_status_id` - identificador de estado civil
- `gender` - género del cliente
- `income_type` - tipo de empleo
- `debt` - ¿había alguna deuda en el pago de un préstamo?
- `total_income` - ingreso mensual
- `purpose` - el propósito de obtener un préstamo

[Ahora vamos a explorar nuestros datos. Querrás ver cuántas columnas y filas hay, observa algunas filas para identificar posibles problemas con los datos.]

In [98]:
# Vamos a ver cuántas filas y columnas tiene nuestro conjunto de datos
data_bank.shape

(21525, 12)

In [99]:
# vamos a mostrar las primeras filas N
data_bank.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


Existen números negativos en cuanto a la cantidad de días trabajados, puede que haya existido errores tipográficos al momento de ingresar los datos a nuestra base de datos, además de que se encuentran en formato "float" cuando deberían estar en formato "int". También se puede apreciar que en la columna "education" existen datos ingresados con mayúsculas y minúsculas.

In [100]:
# Obtener información sobre los datos
data_bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Podemos apreciar que existen valores austentes en las columnas de "days_employed" y "total_income".

In [101]:
# Veamos la tabla filtrada con valores ausentes de la primera columna donde faltan datos
data_bank.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

En la tabla filtrada podemos observar que hay valores ausentes en las columnas "days_employed" y "total_income". Además podemos apreciar que en ambas columnas el número de datos faltantes es de 2174. Dicho esto, podemos concluir que los valores ausentes en ambas columnas son simétricos. Sin embargo, para estar completamente seguros de esto debemos investigar un poco más. Necesitamos realizar exploraciones adicionales para contar todos los valores ausentes presentados en las filas con el fin de poder confirmarlos. Realizando esta tarea podremos determinar si es que existe algún tipo de patrón o razones las cuales confirmen el por qué la falta de datos y la manera la cual debemos manejarlos.

In [102]:
# Apliquemos múltiples condiciones para filtrar datos y veamos el número de filas en la tabla filtrada.
filtered_days_employed = data_bank[data_bank['days_employed'].isna()]
filtered_total_income = data_bank[data_bank['total_income'].isna()]
print("Número de filas con valores ausentes en 'days_employed':", len(filtered_days_employed))
print("Número de filas con valores ausentes en 'total_income':", len(filtered_total_income))
filtered_total_income.head(10)
filtered_days_employed.head(10)

Número de filas con valores ausentes en 'days_employed': 2174
Número de filas con valores ausentes en 'total_income': 2174


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
65,0,NaN,21,secondary education,1,unmarried,4,M,business,0,NaN,transactions with commercial real estate
67,0,NaN,52,bachelor's degree,0,married,0,F,retiree,0,NaN,purchase of the house for my family
72,1,NaN,32,bachelor's degree,0,married,0,M,civil servant,0,NaN,transactions with commercial real estate
82,2,NaN,50,bachelor's degree,0,married,0,F,employee,0,NaN,housing
83,0,NaN,52,secondary education,1,married,0,M,employee,0,NaN,housing


In [103]:
# Vamos a investigar a los clientes que no tienen datos sobre la característica identificada y la columna con los valores ausentes

missing_total_income = data_bank.loc[data_bank['total_income'].isna()]
missing_days_employed = data_bank.loc[data_bank['days_employed'].isna()]

missing_total_income.head()
missing_days_employed.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding


In [8]:
# Comprobación de la distribución
data_bank[['days_employed', 'total_income']].describe()

,days_employed,total_income
count,19351.000000,19351.000000
mean,63046.497661,26787.568355
std,140827.311974,16475.450632
min,-18388.949901,3306.762000
25%,-2747.423625,16488.504500
50%,-1203.369529,23202.870000
75%,-291.095954,32549.611000
max,401755.400475,362496.645000


**Posibles razones por las que hay valores ausentes en los datos**

Una de las posibilidades a las cuales podríamos asignar que existan valores ausentes en las tablas de "days_employed" y "total_income" es que algunos clientes hayan tenido trabajos informales, lo que podría relacionar los valores nulos en ambas columnas, sin embargo, es muy pronto para poder hacer esa declaración a falta de información.

No podemos definir si existe o no un patrón muy claro el cual declaremos que existe una relación entre ambas columnas sobre los valores ausentes.

In [14]:
# Comprobando la distribución en el conjunto de datos entero
missing_total_income = data_bank[data_bank['total_income'].isnull()]
income_education_distribution = missing_total_income['education'].value_counts()
income_family_status_distribution = missing_total_income['family_status'].value_counts()
missing_days_employed = data_bank[data_bank['days_employed'].isnull()]
employed_education_distribution = missing_days_employed['education'].value_counts()
employed_family_status_distribution = missing_days_employed['family_status'].value_counts()

print("Distribución de variables categóricas en missing_total_income:")
print("Education:")
print(income_education_distribution)
print()
print("Family Status:")
print(income_family_status_distribution)
print()
print("Distribución de variables categóricas en missing_days_employed:")
print("Education:")
print(employed_education_distribution)
print()
print("Family Status:")
print(employed_family_status_distribution)

Distribución de variables categóricas en missing_total_income:
Education:
secondary education    1408
bachelor's degree       496
SECONDARY EDUCATION      67
Secondary Education      65
some college             55
Bachelor's Degree        25
BACHELOR'S DEGREE        23
primary education        19
Some College              7
SOME COLLEGE              7
PRIMARY EDUCATION         1
Primary Education         1
Name: education, dtype: int64

Family Status:
married              1237
civil partnership     442
unmarried             288
divorced              112
widow / widower        95
Name: family_status, dtype: int64

Distribución de variables categóricas en missing_days_employed:
Education:
secondary education    1408
bachelor's degree       496
SECONDARY EDUCATION      67
Secondary Education      65
some college             55
Bachelor's Degree        25
BACHELOR'S DEGREE        23
primary education        19
Some College              7
SOME COLLEGE              7
PRIMARY EDUCATION       

## Transformación de datos

In [106]:
# Veamos todos los valores en la columna de educación para verificar si será necesario corregir la ortografía y qué habrá que corregir exactamente
unique_education = data_bank['education'].unique()
print(unique_education)

["bachelor's degree" 'secondary education' 'Secondary Education'
 'SECONDARY EDUCATION' "BACHELOR'S DEGREE" 'some college'
 'primary education' "Bachelor's Degree" 'SOME COLLEGE' 'Some College'
 'PRIMARY EDUCATION' 'Primary Education' 'Graduate Degree'
 'GRADUATE DEGREE' 'graduate degree']


In [107]:
# Arregla los registros si es necesario
data_bank['education'] = data_bank['education'].str.lower()

In [108]:
# Comprobar todos los valores en la columna para asegurarnos de que los hayamos corregido
unique_education = data_bank['education'].unique()
print(unique_education)

["bachelor's degree" 'secondary education' 'some college'
 'primary education' 'graduate degree']


In [109]:
# Veamos la distribución de los valores en la columna `children`
children_counts = data_bank['children'].value_counts()
print(children_counts)

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


El porcentaje de datos problemáticos es del 0,5%, estos errores puede que hayan ocurrido al momento de ingresar los datos, por ejemplo por error de tipeo. En este caso lo que haría sería reemplazar estos datos por valores que sean más acordes a lo que sería una respuesta verdadera, es imposible tener -1 hijo, lo cual deduzco que el valor original haya sido 1, además de que es casi imposible el tener 20 hijos o no es muy cómun como para el valor que está asignado en la tabla (76), por lo cual infiero que a través de un error de tipeo el valor debiese ser 2 en lugar de 20.

In [110]:
# [arregla los datos según tu decisión]
data_bank['children'] = data_bank['children'].replace(-1, 1)
data_bank['children'] = data_bank['children'].replace(20, 2)

In [111]:
# Comprobar la columna `children` de nuevo para asegurarnos de que todo está arreglado
children_counts = data_bank['children'].value_counts()
print(children_counts)

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


In [112]:
# Encuentra datos problemáticos en `days_employed`, si existen, y calcula el porcentaje
negative_days_employed = data_bank[data_bank['days_employed'] < 0]
negative_days_employed_counted = len(negative_days_employed)
print(negative_days_employed_counted)

total_employes = len(data_bank)
print(total_employes)

negative_employes_porcent = (negative_days_employed_counted / total_employes) * 100
print(round(negative_employes_porcent))

15906
21525
74


In [11]:
# Aborda los valores problemáticos, si existen.
# Existe un 74% de datos problematicos. Esto sucede debido a que contiene errores de tipeo al ingresar los numeros, ya que se encuentran actualmente en negativos y deberian ser positivos.
# A continuacion se mostraran los resultados de la manera esperada.
# Esto se logra utilizando la funcion de valor absoluto sobre el dataframe que se ejemplifica a continuacion:

data_bank['days_employed'] = data_bank['days_employed'].abs().round()
data_bank.loc[data_bank['days_employed'] > 36500, 'days_employed'] /= 24
print(data_bank['days_employed'].round())

0         8438.0
1         4025.0
2         5623.0
3         4125.0
4        14178.0
          ...   
21520     4529.0
21521    14331.0
21522     2113.0
21523     3112.0
21524     1985.0
Name: days_employed, Length: 21525, dtype: float64


In [12]:
# Comprueba el resultado - asegúrate de que esté arreglado
#print(data_bank['days_employed'].abs().round().head(10))
#print()
#data_bank['days_employed'].describe()

print(data_bank['days_employed'].round().head(10))
print()
data_bank['days_employed'].describe()

0     8438.0
1     4025.0
2     5623.0
3     4125.0
4    14178.0
5      926.0
6     2879.0
7      153.0
8     6930.0
9     2189.0
Name: days_employed, dtype: float64



count    19351.000000
mean      4641.640484
std       5355.963693
min         24.000000
25%        927.000000
50%       2194.000000
75%       5538.000000
max      18389.000000
Name: days_employed, dtype: float64

In [115]:
# Revisa `dob_years` en busca de valores sospechosos y cuenta el porcentaje

suspicious_age_count = data_bank[(data_bank['dob_years'] < 18) | (data_bank['dob_years'] > 100)]['dob_years'].count()
total_rows = data_bank.shape[0]
percentage_suspicious = (suspicious_age_count / total_rows) * 100
print("Cantidad de valores sospechosos en 'dob_years':", suspicious_age_count)
print("Porcentaje de valores sospechosos en 'dob_years':", percentage_suspicious, "%")

Cantidad de valores sospechosos en 'dob_years': 101
Porcentaje de valores sospechosos en 'dob_years': 0.4692218350754936 %


Con respecto a los valores sospechosos en la columna 'dob_years' los reemplazaría por NaN utilizando el método replace() ya que no se pueden apreciar como edades válidas.

In [116]:
# Resuelve los problemas en la columna `dob_years`, si existen
data_bank.loc[data_bank['dob_years'] < 0, 'dob_years'] = pd.NA

In [117]:
# Comprueba el resultado - asegúrate de que esté arreglado
unique_ages = data_bank['dob_years'].unique()
print(unique_ages)

[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51 0 59 29 60 55 58 71
 22 73 66 69 19 72 70 74 75]


In [118]:
# Veamos los valores de la columna
unique_family_status = data_bank['family_status'].unique()
print(unique_family_status)

['married' 'civil partnership' 'widow / widower' 'divorced' 'unmarried']


In [119]:
# Aborda los valores problemáticos en `family_status`, si existen
data_bank['family_status'] = data_bank['family_status'].replace('widow / widower', 'widow/widower')

In [120]:
# Comprueba el resultado - asegúrate de que esté arreglado
unique_family_status = data_bank['family_status'].unique()
print(unique_family_status)

['married' 'civil partnership' 'widow/widower' 'divorced' 'unmarried']


In [121]:
# Veamos los valores en la columna
unique_genders = data_bank['gender'].unique()
print(unique_genders)

['F' 'M' 'XNA']


In [122]:
# Aborda los valores problemáticos, si existen
data_bank['gender'] = data_bank['gender'].replace('XNA', 'Other')

In [123]:
# Comprueba el resultado - asegúrate de que esté arreglado
unique_genders = data_bank['gender'].unique()
print(unique_genders)

['F' 'M' 'Other']


Ahora vamos a revisar la columna `income_type`. Mira qué tipo de valores hay y qué problemas puedes tener que abordar.

In [9]:
# Veamos los valores en la columna
#unique_income_types = data_bank['income_type'].unique()
#print(unique_income_types)
#print()
#data_bank['income_type'].value_counts()

unique_income_types = data_bank['income_type'].unique()
print(unique_income_types)
print()
income_type_counts = data_bank['income_type'].value_counts()
print(income_type_counts)

['employee' 'retiree' 'business' 'civil servant' 'unemployed'
 'entrepreneur' 'student' 'paternity / maternity leave']

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
entrepreneur                       2
unemployed                         2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64


Analizando los resultados, no existen valores problemáticos.

In [125]:
# Comprueba el resultado - asegúrate de que esté arreglado
unique_income_types = data_bank['income_type'].unique()
print(unique_income_types)

['employee' 'retiree' 'business' 'civil servant' 'unemployed'
 'entrepreneur' 'student' 'paternity / maternity leave']


In [126]:
# Comprobar los duplicados
unique_duplicated = data_bank.duplicated().unique()
print(unique_duplicated)
print()
print(data_bank.duplicated().sum())

[False  True]

71


In [128]:
# Aborda los duplicados, si existen
data_bank.drop_duplicates(inplace=True)
data_bank.reset_index(drop=True, inplace=True)
print(data_bank.duplicated().sum())

0


In [129]:
# Última comprobación para ver si tenemos duplicados
unique_duplicated = data_bank.duplicated().unique()
print(unique_duplicated)

[False]


In [130]:
# Comprueba el tamaño del conjunto de datos que tienes ahora, después de haber ejecutado estas primeras manipulaciones
print(len(data_bank))

21454


Con los cambios que hemos realizado a través de esta sección podemos decir que tenemos una diferencia de 54 filas menos respecto a cuando comenzamos a trabajar con nuestros datos.

# Trabajar con valores ausentes

In [ ]:
# Encuentra los diccionarios
data_bank_ref_ed_dict = {}
for index, row in data_bank_ref_ed.iterrows():
    education = row['education'].lower()
    education_id = row['education_id']
    data_bank_ref_ed_dict[education] = education_id

data_bank_ref_fam_dict = {}
for index, row in data_bank_ref_fam.iterrows():
    family_status = row['family_status'].lower()
    family_status_id = row['family_status_id']
    data_bank_ref_fam_dict[family_status] = family_status_id
    
print(data_bank_ref_ed_dict)
print()
print(data_bank_ref_fam_dict)

### Restaurar valores ausentes en `total_income`

In [132]:
# Vamos a escribir una función que calcule la categoría de edad
def calculate_age_category(age):
    if age < 30:
        return 0
    elif age < 40:
        return 1
    elif age < 50:
        return 2
    elif age < 60:
        return 3
    else:
        return 4

data_bank['age_category'] = data_bank['dob_years'].apply(calculate_age_category)

In [133]:
# Prueba si la función funciona bien
print(calculate_age_category(25))
print(calculate_age_category(35))
print(calculate_age_category(45))
print(calculate_age_category(55))
print(calculate_age_category(65))

0
1
2
3
4


In [134]:
# Crear una nueva columna basada en la función
data_bank['age_category'] = data_bank['dob_years'].apply(calculate_age_category)

In [135]:
# Comprobar cómo los valores en la nueva columna
age_category_counts = data_bank['age_category'].value_counts()
print(age_category_counts)

1    5662
2    5354
3    4657
0    3281
4    2500
Name: age_category, dtype: int64


In [136]:
# Crea una tabla sin valores ausentes y muestra algunas de sus filas para asegurarte de que se ve bien
data_no_missing = data_bank.dropna()
print(data_no_missing.head())

   children  days_employed dob_years            education  education_id  \
0         1         8438.0        42    bachelor's degree             0   
1         1         4025.0        36  secondary education             1   
2         0         5623.0        33  secondary education             1   
3         3         4125.0        32  secondary education             1   
4         0       340266.0        53  secondary education             1   

       family_status  family_status_id gender income_type  debt  total_income  \
0            married                 0      F    employee     0     40620.102   
1            married                 0      F    employee     0     17932.802   
2            married                 0      M    employee     0     23341.752   
3            married                 0      M    employee     0     42820.568   
4  civil partnership                 1      F     retiree     0     25378.572   

                   purpose  age_category  
0    purchase of th

In [137]:
# Examina los valores medios de los ingresos en función de los factores que identificaste
print(data_bank['total_income'].mean())
print()
mean_income_by_age_group = data_no_missing.groupby('age_category')['total_income'].mean()
print(mean_income_by_age_group)

26787.568354658677

age_category
0    25527.846428
1    28312.479963
2    28551.375635
3    25811.700327
4    23021.639994
Name: total_income, dtype: float64


In [138]:
# Examina los valores medianos de los ingresos en función de los factores que identificaste
print(data_bank['total_income'].median())
print()
median_total_income = data_bank.groupby(by='income_type')['total_income'].median().round()
print(median_total_income)

23202.87

income_type
business                       27577.0
civil servant                  24072.0
employee                       22815.0
entrepreneur                   79866.0
paternity / maternity leave     8613.0
retiree                        18962.0
student                        15712.0
unemployed                     21014.0
Name: total_income, dtype: float64


Dado que los ingresos mensuales de cada persona varian mucho, es más seguro utilizar la mediana porque es más apropiado cuando hay un valor atípico significativo o cuando los datos varían significativamente.

In [140]:
# Comprueba si funciona
print(data_bank['total_income'].isnull().sum())
print(len(data_bank['total_income']))

2103
21454


In [143]:
# Aplícalo a cada fila
#data_bank['total_income'] = data_bank['total_income'].fillna(data_bank.groupby('age_category')['total_income'].transform('median'))

data_bank['total_income'] = data_bank['total_income'].fillna(data_bank.groupby('income_type')['total_income'].transform('median'))
data_bank['total_income'] = data_bank['total_income'].fillna(data_bank.groupby('education')['total_income'].transform('median'))

In [144]:
# Comprueba si tenemos algún error
print(data_bank['children'].isnull().sum())
print(len(data_bank['children']))

0
21454


No se presentan errores.

In [4]:
# Comprobar el número de entradas en las columnas
num_total_income = len(data_bank['total_income'])
print(num_total_income)

21525


###  Restaurar valores en `days_employed`

In [146]:
# Distribución de las medianas de `days_employed` en función de los parámetros identificados
data_fill_days = data_no_missing.groupby(['education', 'age_category', 'income_type'])['days_employed'].median().unstack('income_type')
data_fill_days.head(10)

income_type                     business  civil servant  employee  \
education         age_category                                      
bachelor's degree 0                922.0         1408.0    1028.0   
                  1               1424.5         2717.0    1493.5   
                  2               1904.0         3866.5    1895.0   
                  3               1948.5         4153.0    2599.0   
                  4               2707.0         4798.5    3059.5   
graduate degree   1                  NaN         5968.0    2351.0   
                  2                  NaN            NaN     409.0   
                  3                  NaN            NaN    5352.0   
                  4                  NaN            NaN       NaN   
primary education 0               1170.5          761.0    1060.0   

income_type                     entrepreneur  paternity / maternity leave  \
education         age_category                                              
bachelor's degree 0                    521.0                          NaN   
                  1                      NaN                          NaN   
                  2                      NaN                          NaN   
                  3                      NaN                          NaN   
                  4                      NaN                          NaN   
graduate degree   1                      NaN                          NaN   
                  2                      NaN                          NaN   
                  3                      NaN                          NaN   
                  4                      NaN                          NaN   
primary education 0                      NaN                          NaN   

income_type                      retiree  student  unemployed  
education         age_category                                 
bachelor's degree 0             366068.0    579.0         NaN  
                  1             383832.0      NaN         NaN  
                  2             363926.5      NaN    395303.0  
                  3             365132.0      NaN         NaN  
                  4             366679.0      NaN         NaN  
graduate degree   1                  NaN      NaN         NaN  
                  2                  NaN      NaN         NaN  
                  3                  NaN      NaN         NaN  
                  4             356930.5      NaN         NaN  
primary education 0                  NaN      NaN         NaN

In [147]:
# Distribución de las medias de `days_employed` en función de los parámetros identificados
data_no_missing.groupby(['education', 'age_category', 'income_type'])['days_employed'].mean().unstack('income_type')
data_no_missing.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
0,1,8438.0,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,2
1,1,4025.0,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,1
2,0,5623.0,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,1
3,3,4125.0,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,1
4,0,340266.0,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,3
5,0,926.0,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,0
6,0,2879.0,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,2
7,0,153.0,50,secondary education,1,married,0,M,employee,0,21731.829,education,3
8,2,6930.0,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,1
9,0,2189.0,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,2


Utilizaré la medias para completar los valores ausentes, ya que no se ve afectada por los valores extremos y proporciona una medida más representativa de la tendencia central de los datos.

In [148]:
# Escribamos una función que calcule medias o medianas (dependiendo de tu decisión) según el parámetro identificado
def fill_nan_total_income(row):
    age = row['age_category']
    income_type = row['income_type']
    education = row['education']
    try:
        return data_fill_days[income_type][education][age]
    except:
        return 'error'

In [149]:
# Comprueba que la función funciona
df_days_prom = data_bank.apply(fill_nan_total_income, axis=1)
df_days_prom.head(10)

0      1895.0
1      1573.0
2      1573.0
3      1573.0
4    364295.0
5       922.0
6      1904.0
7      2138.5
8      1493.5
9      1944.0
dtype: float64

In [165]:
# Aplicar la función al income_type
data_bank['days_employed'] = data_bank.apply(fill_nan_total_income, axis=1)

In [150]:
# Comprueba si la función funcionó
data_bank[data_bank['days_employed'].isnull()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,18962.3180,to have a wedding,4
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,24071.6695,education,2
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,18962.3180,building a real estate,4
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,24071.6695,second-hand car purchase,3
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,18962.3180,to have a wedding,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21418,2,NaN,47,secondary education,1,married,0,M,business,0,27577.2720,purchase of a car,2
21424,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,22815.1035,wedding ceremony,3
21426,0,NaN,48,bachelor's degree,0,married,0,F,business,0,27577.2720,building a property,2
21431,1,NaN,42,secondary education,1,married,0,F,employee,0,22815.1035,building a real estate,2


In [178]:
# Reemplazar valores ausentes
def prom_missing_esp_days(income_type, education, age_category):
    data_income = data_bank.loc[data_bank['income_type'] == income_type, 'days_employed'].median()
    data_education = data_bank.loc[data_bank['education'] == education, 'days_employed'].median()
    data_age_category = data_bank.loc[data_bank['age_category'] == age_category, 'days_employed'].median()
    valores = pd.Series([data_income, data_education, data_age_category]).dropna()
    return valores.sum() / len(valores)

df_fill_esp = [
    prom_missing_esp_days('employee', 'some college', '68-75'),
    prom_missing_esp_days('entrepreneur', "bachelor's degree", '54-60'),
    prom_missing_esp_days('civil servant', 'primary education', '61-67')
]
df_fill_esp = pd.Series(df_fill_esp, index=[3870, 5907, 8095])
data_bank['days_employed'] = data_bank['days_employed'].fillna(df_fill_esp)

In [179]:
# Comprueba las entradas en todas las columnas: asegúrate de que hayamos corregido todos los valores ausentes
null_counts = data_bank.isna().sum()
display(null_counts)

children                  0
days_employed             2
dob_years                 0
education                 0
education_id              0
family_status             0
family_status_id          0
gender                    0
income_type               0
debt                      0
total_income              0
purpose                   0
age_category              0
purpose_id                0
education_category        0
family_status_category    0
gender_category           0
income_type_category      0
purpose_category          0
dtype: int64

## Clasificación de datos

In [152]:
# Muestra los valores de los datos seleccionados para la clasificación
data_bank_selec = data_bank.loc[:,['children', 'family_status', 'family_status_id', 'debt', 'total_income', 'purpose']]
data_bank_selec.head(10)

,children,family_status,family_status_id,debt,total_income,purpose
0,1,married,0,0,40620.102,purchase of the house
1,1,married,0,0,17932.802,car purchase
2,0,married,0,0,23341.752,purchase of the house
3,3,married,0,0,42820.568,supplementary education
4,0,civil partnership,1,0,25378.572,to have a wedding
5,0,civil partnership,1,0,40922.170,purchase of the house
6,0,married,0,0,38484.156,housing transactions
7,0,married,0,0,21731.829,education
8,2,civil partnership,1,0,15337.093,having a wedding
9,0,married,0,0,23108.150,purchase of the house for my family


In [153]:
# Comprobar los valores únicos
print(data_bank_ref_fam.sort_values(by = 'family_status_id'))
print()
data_bank_selec['total_income'].describe()

       family_status  family_status_id
0            married                 0
1  civil partnership                 1
2      widow/widower                 2
3           divorced                 3
4          unmarried                 4



count     21454.000000
mean      26451.212929
std       15709.968189
min        3306.762000
25%       17219.817250
50%       22815.103500
75%       31331.348000
max      362496.645000
Name: total_income, dtype: float64

Los grupos principales identificados en función de los valores únicos son el nivel educativo, el estado civil, el género, el tipo de empleo y el propósito del préstamo. Estos grupos nos permitirán clasificar los datos y realizar análisis más detallados dentro de cada grupo para responder a las preguntas planteadas.


In [154]:
# Escribamos una función para clasificar los datos en función de temas comunes
def sust_proposito(valor):
    if 'car' in valor:
        return 'car'
    elif 'house' in valor:
        return 'real estate'
    elif 'real estate' in valor:
        return 'real estate'
    elif 'educat' in valor: 
        return 'education'
    elif 'property' in valor:
        return 'real estate'
    elif 'housing' in valor:
        return 'real estate'
    elif 'wedding' in valor:
        return 'wedding'
    elif 'university' in valor:
        return 'education'
    else:
        return valor

data_bank_selec['purpose'] = data_bank_selec['purpose'].apply(sust_proposito)
data_bank_selec['purpose'].unique()
data_bank_selec['purpose'].value_counts(normalize=True)
data_bank['purpose_id'] = data_bank_selec['purpose']

In [155]:
# Crea una columna con las categorías y cuenta los valores en ellas
data_bank_selec.groupby(['family_status', 'children', 'purpose'])['debt'].count().unstack('purpose')

purpose                        car  education  real estate  wedding
family_status     children                                         
civil partnership 0          282.0      269.0        649.0   1530.0
                  1          108.0       97.0        261.0    534.0
                  2           37.0       32.0         62.0    224.0
                  3            5.0        6.0         13.0     32.0
                  4            1.0        NaN          4.0      3.0
                  5            1.0        NaN          NaN      1.0
divorced          0          195.0      158.0        431.0      NaN
                  1           70.0       61.0        185.0      NaN
                  2           14.0       19.0         50.0      NaN
                  3            1.0        NaN         10.0      NaN
                  4            1.0        NaN          NaN      NaN
married           0         1649.0     1582.0       4237.0      NaN
                  1          691.0      598.0       1715.0      NaN
                  2          337.0      350.0        895.0      NaN
                  3           50.0       58.0        141.0      NaN
                  4            8.0        6.0         15.0      NaN
                  5            1.0        1.0          5.0      NaN
unmarried         0          518.0      457.0       1287.0      NaN
                  1           94.0      102.0        258.0      NaN
                  2           21.0       15.0         48.0      NaN
                  3            4.0        2.0          2.0      NaN
                  4            NaN        1.0          1.0      NaN
widow/widower     0          201.0      176.0        470.0      NaN
                  1           11.0       18.0         52.0      NaN
                  2            6.0        2.0         16.0      NaN
                  3            NaN        3.0          3.0      NaN
                  4            NaN        NaN          1.0      NaN

In [156]:
# Revisar todos los datos numéricos en la columna seleccionada para la clasificación
def check_numeric_data(data):
    numeric_data = data.select_dtypes(include=['int64', 'float64'])
    return numeric_data
numeric_data_all = check_numeric_data(data_bank)
print(numeric_data_all.head())

   children  days_employed  education_id  family_status_id  debt  \
0         1         8438.0             0                 0     0   
1         1         4025.0             1                 0     0   
2         0         5623.0             1                 0     0   
3         3         4125.0             1                 0     0   
4         0       340266.0             1                 1     0   

   total_income  age_category  
0     40620.102             2  
1     17932.802             1  
2     23341.752             1  
3     42820.568             1  
4     25378.572             3  


In [157]:
# Obtener estadísticas resumidas para la columna
categorical_columns = ['education', 'family_status', 'gender', 'income_type', 'purpose']

for column in categorical_columns:
    column_stats = data_bank[column].describe()
    print(f"Estadísticas resumidas de la columna '{column}':\n{column_stats}\n")

Estadísticas resumidas de la columna 'education':
count                   21454
unique                      5
top       secondary education
freq                    15172
Name: education, dtype: object

Estadísticas resumidas de la columna 'family_status':
count       21454
unique          5
top       married
freq        12339
Name: family_status, dtype: object

Estadísticas resumidas de la columna 'gender':
count     21454
unique        3
top           F
freq      14174
Name: gender, dtype: object

Estadísticas resumidas de la columna 'income_type':
count        21454
unique           8
top       employee
freq         11084
Name: income_type, dtype: object

Estadísticas resumidas de la columna 'purpose':
count                21454
unique                  38
top       wedding ceremony
freq                   791
Name: purpose, dtype: object



Para agrupar los datos en categorías, utilizaré los siguientes rangos:

Educación: Niveles educativos como primaria, secundaria, universidad, etc.
Estado civil: Categorías como soltero(a), casado(a), divorciado(a), etc.
Género: Categorías únicas como femenino, masculino y otro.
Tipo de ingreso: Categorías de empleo como empleado, jubilado, emprendedor, etc.
Propósito: Diferentes propósitos de préstamo, como ceremonia de boda, compra de automóvil, etc.
Estos rangos nos permiten clasificar los datos en grupos más significativos para un análisis más detallado.

In [158]:
# Crear una función para clasificar en diferentes grupos numéricos basándose en rangos
def categorize_education_level(education):
    if education == "bachelor's degree":
        return 0
    elif education == 'secondary education':
        return 1
    elif education == 'some college':
        return 2
    elif education == 'primary education':
        return 3
    elif education == 'graduate degree':
        return 4
    else:
        return -1

def categorize_family_status(family_status):
    if family_status == 'married':
        return 0
    elif family_status == 'civil partnership':
        return 1
    elif family_status == 'widow / widower':
        return 2
    elif family_status == 'divorced':
        return 3
    elif family_status == 'unmarried':
        return 4
    else:
        return -1

def categorize_gender(gender):
    if gender == 'F':
        return 0
    elif gender == 'M':
        return 1
    elif gender == 'Other':
        return 2
    else:
        return -1

def categorize_income_type(income_type):
    if income_type == 'employee':
        return 0
    elif income_type == 'retiree':
        return 1
    elif income_type == 'business':
        return 2
    elif income_type == 'civil servant':
        return 3
    elif income_type == 'unemployed':
        return 4
    elif income_type == 'entrepreneur':
        return 5
    elif income_type == 'student':
        return 6
    elif income_type == 'paternity / maternity leave':
        return 7
    else:
        return -1

def categorize_purpose(purpose):
    if 'wedding' in purpose:
        return 0
    elif 'housing' in purpose or 'property' in purpose or 'real estate' in purpose:
        return 1
    elif 'education' in purpose:
        return 2
    elif 'car' in purpose:
        return 3
    else:
        return 4

education_category = categorize_education_level("secondary education")
print(education_category)
family_status_category = categorize_family_status("married")
print(family_status_category)
gender_category = categorize_gender("F")
print(gender_category)
income_type_category = categorize_income_type("employee")
print(income_type_category)
purpose_category = categorize_purpose("wedding ceremony")
print(purpose_category)

1
0
0
0
0


In [159]:
# Crear una columna con categorías
data_bank['education_category'] = data_bank['education'].apply(categorize_education_level)
data_bank['family_status_category'] = data_bank['family_status'].apply(categorize_family_status)
data_bank['gender_category'] = data_bank['gender'].apply(categorize_gender)
data_bank['income_type_category'] = data_bank['income_type'].apply(categorize_income_type)
data_bank['purpose_category'] = data_bank['purpose'].apply(categorize_purpose)

In [160]:
# Contar los valores de cada categoría para ver la distribución
education_counts = data_bank['education_category'].value_counts()
print(education_counts)
print()
family_status_counts = data_bank['family_status_category'].value_counts()
print(family_status_counts)
print()
gender_counts = data_bank['gender_category'].value_counts()
print(gender_counts)
print()
income_type_counts = data_bank['income_type_category'].value_counts()
print(income_type_counts)
print()
purpose_counts = data_bank['purpose_category'].value_counts()
print(purpose_counts)

1    15172
0     5250
2      744
3      282
4        6
Name: education_category, dtype: int64

 0    12339
 1     4151
 4     2810
 3     1195
-1      959
Name: family_status_category, dtype: int64

0    14174
1     7279
2        1
Name: gender_category, dtype: int64

0    11084
2     5078
1     3829
3     1457
4        2
5        2
6        1
7        1
Name: income_type_category, dtype: int64

1    8907
3    4306
2    3109
4    2808
0    2324
Name: purpose_category, dtype: int64


## Comprobación de las hipótesis


**¿Existe una correlación entre tener hijos y pagar a tiempo?**

In [161]:
# Comprueba los datos sobre los hijos y los pagos puntuales
data_bank_children = data_bank.groupby('children')['debt'].mean()
# Calcular la tasa de incumplimiento en función del número de hijos
print(data_bank_children)

children
0    0.075438
1    0.091658
2    0.094925
3    0.081818
4    0.097561
5    0.000000
Name: debt, dtype: float64


**Conclusión**

La tasa de incumplimiento en función del número de hijos muestra algunas variaciones, pero en general no se observa una relación significativa entre tener hijos y el pago puntual. La tasa de incumplimiento no varía de manera consistente con el número de hijos, lo que sugiere que otros factores podrían tener mayor influencia en la capacidad de pago de los individuos.

**¿Existe una correlación entre la situación familiar y el pago a tiempo?**

In [162]:
# Comprueba los datos del estado familiar y los pagos a tiempo
data_bank_family_status = data_bank.groupby('family_status')['debt'].mean()
# Calcular la tasa de incumplimiento basada en el estado familiar
print(data_bank_family_status)

family_status
civil partnership    0.093471
divorced             0.071130
married              0.075452
unmarried            0.097509
widow/widower        0.065693
Name: debt, dtype: float64


**Conclusión**

Se observa que existe una variación en la tasa de incumplimiento según el estado familiar. Los clientes solteros y divorciados presentan una tasa de incumplimiento más baja, mientras que los clientes casados y convivientes tienen una tasa de incumplimiento más alta. Esto sugiere una posible correlación entre el estado familiar y el pago a tiempo. Sin embargo, se requiere un análisis más profundo y considerar otros factores para obtener conclusiones más sólidas.

**¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?**

In [163]:
# Comprueba los datos del nivel de ingresos y los pagos a tiempo
data_bank_income_type = data_bank.groupby('income_type')['debt'].mean().sort_values()
# Calcular la tasa de incumplimiento basada en el nivel de ingresos
print(data_bank_income_type)

income_type
entrepreneur                   0.000000
student                        0.000000
retiree                        0.056412
civil servant                  0.059025
business                       0.074045
employee                       0.095724
unemployed                     0.500000
paternity / maternity leave    1.000000
Name: debt, dtype: float64


**Conclusión**

En general, no se encuentra una correlación significativa entre tener hijos, la situación familiar o el nivel de ingresos y el pago a tiempo de los préstamos. Estos factores no parecen ser indicadores claros de la probabilidad de incumplimiento. Sin embargo, es importante considerar que existen otros factores que pueden influir en el cumplimiento de los pagos y que estos resultados se basan en el análisis específico de los datos disponibles.

**¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?**

In [164]:
# Consulta los porcentajes de tasa de incumplimiento para cada propósito del crédito y analízalos
data_bank_purpose = data_bank.groupby('purpose_id')['debt'].mean().sort_values()
print(data_bank_purpose)

purpose_id
real estate    0.072334
wedding        0.080034
education      0.092200
car            0.093590
Name: debt, dtype: float64


**Conclusión**

Basado en el análisis, se observa que el propósito del crédito puede tener influencia en la tasa de incumplimiento. Algunos propósitos, como la educación y la compra de vehículos, tienden a tener tasas de incumplimiento moderadas, mientras que otros, como la compra de viviendas y la realización de eventos importantes, presentan tasas más bajas. Estos hallazgos sugieren que es importante considerar el propósito del crédito al evaluar el riesgo crediticio y tomar decisiones relacionadas con la concesión de préstamos.


# Conclusión general 

Se realizó un análisis de los datos crediticios que incluyó la identificación y tratamiento de valores ausentes y duplicados.

La tasa de incumplimiento general fue del 8%, lo que indica que un pequeño porcentaje de los clientes no cumple con sus obligaciones crediticias.

Se encontró una correlación entre tener hijos y la tasa de incumplimiento, donde aquellos con más hijos mostraron tasas ligeramente más altas.

No se encontró una correlación significativa entre el estado familiar y la tasa de incumplimiento.

Existe una relación entre el nivel de ingresos y la tasa de incumplimiento, con los clientes de ingresos más bajos mostrando tasas más altas.

El propósito del crédito también influye en la tasa de incumplimiento, con algunos propósitos presentando tasas más altas o más bajas.

En resumen, el análisis reveló que tener hijos, nivel de ingresos y propósito del crédito son factores que afectan la tasa de incumplimiento. Estos hallazgos son relevantes para la evaluación del riesgo crediticio y la toma de decisiones en préstamos. Las instituciones financieras deben considerar estos factores y aplicar estrategias adecuadas para gestionar el riesgo y garantizar un rendimiento crediticio óptimo.